In [ ]:
import os
import numpy as np
import pandas as pd
import PIL
from PIL import Image
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pickle
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler, Subset, Dataset
from sklearn.model_selection import KFold
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, ConfusionMatrixDisplay

PIL.Image.MAX_IMAGE_PIXELS = 933120000000

LR = 5e-4
EPOCHS = 100
BATCH_SIZE = 4
N_SPLITS = 5
N_ACCUMULATE = 16
MODEL_NAME = 'resnetaa50d.d_in12k'
Image_size = 1024
WEIGHT_DECAY = 1e-4
num_classes = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())


# //
# //                       _oo0oo_
# //                      o8888888o
# //                      88" . "88
# //                      (| -_- |)
# //                      0\  =  /0
# //                    ___/`---'\___
# //                  .' \\|     |// '.
# //                 / \\|||  :  |||// \
# //                / _||||| -:- |||||- \
# //               |   | \\\  -  /// |   |
# //               | \_|  ''\---/''  |_/ |
# //               \  .-\__  '-'  ___/-. /
# //             ___'. .'  /--.--\  `. .'___
# //          ."" '<  `.___\_<|>_/___.' >' "".
# //         | | :  `- \`.;`\ _ /`;.`/ - ` : | |
# //         \  \ `_.   \_ __\ /__ _/   .-` /  /
# //     =====`-.____`.___ \_____/___.-`___.-'=====
# //                       `=---='
# //
# //
# //     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# //
# //               佛祖保佑         永无BUG
# //

In [ ]:
# Load the saved .npy files
X_train_path = '/kaggle/input/cs640-test-npy/X_train_1024 (1).npy'  # replace with the actual path

# Check if the files exist before loading them
if os.path.exists(X_train_path):
    X_train = np.load(X_train_path)
else:
    print("Files not found. Please check the paths.")

# Verify the shape of arrays
print(f"X_train shape: {X_train.shape}")

In [ ]:
print(f"X_train shape: {X_train.shape}")
X_train_tensor = torch.tensor(X_train).float()
plt.imshow(X_train_tensor[0]/255)
plt.show()

In [ ]:
X_train_tensor = X_train_tensor.view(-1, Image_size, Image_size, 3)

In [ ]:
torch.manual_seed(0)
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(), 
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    # transforms.CenterCrop((224, 224)),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
dataset = TensorDataset(X_train_tensor)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)


model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=5)
# Load the trained model
model_checkpoint = torch.load('/kaggle/input/resnetaa50d-checkpoint/model_checkpoint.pth')
model.load_state_dict(model_checkpoint)
model.to(device)
model.eval()

all_preds = []
with torch.no_grad():
    for batch in loader:  # 此处的 batch 是一个包含一个元素的元组
        inputs = batch[0]  # 解包元组以获取输入
        inputs = inputs.to(device)
        inputs = inputs / 255  # 缩放到 0-1 范围
        inputs = inputs.permute(0, 3, 1, 2)  # 调整通道顺序
        inputs = image_transforms(inputs)  # 应用其他图像变换
        outputs = model(inputs)

        # 获取预测结果
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.view(-1).cpu().numpy())

print(all_preds)



import csv

# 假设 all_preds 包含模型的预测标签
# 例如：all_preds = [0, 1, 2, ...]

# 创建并写入 CSV 文件
with open('/kaggle/working/predictions.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_id', 'label'])

    # 循环迭代 all_preds，索引即为 image_id
    for image_id, label in enumerate(all_preds):
        writer.writerow([image_id, label])

print("预测已成功保存到 /kaggle/working/predictions.csv 文件中")

In [ ]:
# Load the saved .npy files
X_train_path = '/kaggle/input/cs640-npy/X_train_1024.npy'  # replace with the actual path
Y_train_path = '/kaggle/input/cs640-npy/Y_train_1024.npy'  # replace with the actual path

# Check if the files exist before loading them
if os.path.exists(X_train_path) and os.path.exists(Y_train_path):
    X_train = np.load(X_train_path)
    Y_train = np.load(Y_train_path)
else:
    print("Files not found. Please check the paths.")

# Verify the shape of arrays
print(f"X_train shape: {X_train.shape}")
print(f"Y_train shape: {Y_train.shape}")

In [ ]:
X_train_tensor = torch.tensor(X_train).float()
Y_train_tensor = torch.from_numpy(Y_train).long()  # assume Y_train contains integer class labels

X_train_tensor = X_train_tensor.view(-1, Image_size, Image_size, 3)
dataset = TensorDataset(X_train_tensor, Y_train_tensor)
# Split the dataset into a training set and a validation set
train_idx, val_idx = train_test_split(
    np.arange(len(X_train)),
    test_size=0.2,  # reserve 20% for validation
    random_state=43,  # guarantee repeatability
    stratify=Y_train  # keep data distribution consistent
)

train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)

train_labels = [dataset.tensors[1][i].item() for i in train_idx]
val_labels = [dataset.tensors[1][i].item() for i in val_idx]

train_images = [dataset[i][0] for i in train_idx]
val_images = [dataset[i][0] for i in val_idx]

plt.imshow(train_images[0]/255)
plt.show()
print(train_images[0].shape)

Y_train = np.array(Y_train)  # convert to numpy array if not already
class_sample_count = np.array([len(np.where(Y_train[train_idx] == t)[0]) for t in np.unique(Y_train[train_idx])])

weights = 1. / class_sample_count
samples_weights = np.array([weights[t] for t in Y_train[train_idx]])

# Convert to a PyTorch tensor
samples_weights = torch.from_numpy(samples_weights)
samples_weights = samples_weights.double()

# Create a sampler for weighted sampling
sampler = WeightedRandomSampler(samples_weights, len(samples_weights))

# Create data loaders for training and validation
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(len(np.unique(Y_train)))

first_batch_images, first_batch_labels = next(iter(train_loader))

plt.imshow(first_batch_images[0]/255)
plt.show()

In [ ]:
torch.manual_seed(2)
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(), 
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    # transforms.CenterCrop((224, 224)),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=5)
# Load the trained model
model_checkpoint = torch.load('/kaggle/input/resnetaa50d-checkpoint/model_checkpoint.pth')
model.load_state_dict(model_checkpoint)
model.to(device)
model.eval()

val_preds = []
val_targets = []
val_loss = 0.0
val_correct = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs/255
        inputs = inputs.permute(0, 3, 1, 2)
        inputs = image_transforms(inputs)
        outputs = model(inputs)
        
        _, preds = torch.max(outputs, 1)
        val_preds.extend(preds.view(-1).cpu().numpy())
        val_targets.extend(labels.view(-1).cpu().numpy())
        
conf_matrix = confusion_matrix(val_targets, val_preds)
f1 = f1_score(val_targets, val_preds, average='weighted')
balanced_acc = balanced_accuracy_score(val_targets, val_preds)

print("confusion matrix:\n", conf_matrix)
print("F1 Score: {:.4f}".format(f1))
print("Balanced accuracy: {:.4f}".format(balanced_acc))